### Importar Librerias

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, round, hour, dayofweek

### Crea una Session en PySpark

In [2]:
spark = SparkSession.builder.appName("ProcesarDatosNYC").getOrCreate()

### Leer parquet crudo

In [3]:
df = spark.read.parquet("../data/processed/yellow_2019_raw.parquet")

### Calcular duración en minutos

In [4]:
df = df.withColumn("pickup_ts", unix_timestamp("tpep_pickup_datetime"))
df = df.withColumn("dropoff_ts", unix_timestamp("tpep_dropoff_datetime"))
df = df.withColumn("duracion_min", round((df["dropoff_ts"] - df["pickup_ts"]) / 60, 2))

### Enriquecer con hora y día

In [5]:
df = df.withColumn("hora_recogida", hour("tpep_pickup_datetime"))
df = df.withColumn("dia_semana", dayofweek("tpep_pickup_datetime"))

### Filtrar viajes inválidos

In [6]:
df = df.filter((df["duracion_min"] > 1) & (df["duracion_min"] < 180))

### Guardar dataset procesado

In [7]:
df.write.mode("overwrite").parquet("../data/processed/yellow_2019_procesado.parquet")

print("✅ Datos procesados y guardados")

ERROR! Session/line number was not unique in database. History logging moved to new session 7


ConnectionRefusedError: [Errno 111] Connection refused